In [1]:
using Flux
using Flux: onehot, onecold, logitcrossentropy #, throttle, @epochs
using MLDatasets
using Random
using JLD2
using Statistics
using MLJBase
using BSON: @load # for load weights
using Plots
using Zygote
using Distributions

In [2]:
train_data = JLD2.load_object("data/train_data.jld2");
test_data = JLD2.load_object("data/test_data.jld2");
x_train = JLD2.load_object("data/x_train.jld2");
y_train = JLD2.load_object("data/y_train.jld2");
x_test  = JLD2.load_object("data/x_test.jld2");
y_test  = JLD2.load_object("data/y_test.jld2");

In [3]:
rng =Random.default_rng()
FoodNumber = 1000
limit = 50
maxCycle = 50
a = 1

1

In [4]:
model = Chain(Dense(19, 2*19, sigmoid), Dense(2*19, 2))#define the model 
θ, re = Flux.destructure(model);

In [5]:
globalFoodSourceFitness = zeros(Float64, FoodNumber)
globalfoodSourceCount = zeros(Float64, FoodNumber)
globalWeights = [Vector{Float64}(undef,size(θ)[1]) for _ in 1:FoodNumber];
lowerBound = -1ones(Float64, size(θ)[1])
upperBound = 1ones(Float64, size(θ)[1]);
preCal = upperBound-lowerBound;
Dimension = size(θ)[1];

In [6]:
function initABC()
    for i in 1:FoodNumber
        for j in 1:Dimension
            newpar = lowerBound[j] + rand()*preCal[j];
            if newPar < lowerBound[j]
                newPar = lowerBound[j];
            elseif newPar > upperBound[j]
                newPar = upperBound[j];
            end
            globalWeights[i][j] = newpar
        end
        globalFoodSourceFitness[i] = 0
        globalfoodSourceCount[i] = 0
    end
end

initABC (generic function with 1 method)

In [8]:
function updateFoodSource(index, newSource)
    localFitness = calculateFitness(train_data, newSource)
    if localFitness > globalFoodSourceFitness[index]
        globalFoodSourceFitness[index] = localFitness
        globalWeights[index] = newSource
        globalfoodSourceCount[index] = 0
    else
        globalfoodSourceCount[index] = globalfoodSourceCount[index]+1
    end
end


updateFoodSource (generic function with 1 method)

In [ ]:
function getBest()
    best = 1
    for i = 2:FoodNumber
        if globalFoodSourceFitness[i] > globalFoodSourceFitness[best]
            best = i
        end
    end
    return best
end

In [ ]:
function employedBeeFindNew()
    for i in 1:FoodNumber
        FindNewFoodSource(i)
    end
end

In [9]:
function FindNewFoodSource(index)
    j = index
    while j == index
        j = rand(1:FoodNumber)
    end
    newSource = zeros(Float64,Dimension);
    for i in 1:Dimension
        newPar = globalWeights[index][i] + a*rand(Uniform(-1,1))*(globalWeights[index][i]-globalWeights[j][i])
        if newPar < lowerBound[j]
            newPar = lowerBound[j];
        elseif newPar > upperBound[j]
            newPar = upperBound[j];
        end
        newSource[i] = newPar
    end
    updateFoodSource(index, newSource)
    checkRetire()
end



FindNewFoodSource (generic function with 1 method)

In [ ]:
function scoutBeeFindNew()
    checkRetire()
end

In [ ]:
function checkRetire()
    for i in 1:FoodNumber
        if globalfoodSourceCount[i] > limit
            FindNewFoodSource(i, globalWeights)
        end
    end
end

In [10]:
function rouletteWheelSelection()
    fitnessSum = sum(globalFoodSourceFitness)
    fitnessProb = globalFoodSourceFitness/fitnessSum
    cumProb = cumsum(fitnessProb)
    randNum = rand()
    for i in 1:FoodNumber
        if randNum <= cumProb[i]
            return i
        end
    end
end

rouletteWheelSelection (generic function with 1 method)

In [ ]:
function lookoutFindNew()
    for i in 1:FoodNumber
        index = rouletteWheelSelection()
        FindNewFoodSource(index, globalWeights)
    end
end